### Clustering using Edit Distance
Trying to implement strand isolation using purely Edit Distance. Want to select the same strands together after signatures using some similarity metric

In [1]:
%load_ext autoreload
%autoreload 2

In [70]:
import numpy as np
import aligned_clustering
from utils import get_fastq_records, load_json_file, get_original_strands, get_badread_strand_id, get_recovery_percentage, create_random_strand, len_histogram, get_sort_by_sublists_length
import Levenshtein
import random
from Levenshtein import ratio, distance
from collections import Counter
from seq_stat import align
import matplotlib.pyplot as plt
import heirarchal_clustering
import strand_filtering
from tqdm.notebook import tqdm
import utils

In [43]:
#records_original = get_fastq_records(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_no_adapters.fastq")
records = utils.postprocess_badread_sequencing_data(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_adapters.fastq\reads_adapters.fastq", reverse_oriented=False)

296it [00:00, 21007.36it/s]


In [44]:
original_strand_ids, coupling_rates, capping_flags, original_strands = get_original_strands(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\original_strands.txt")

strand_ids_synthesized = load_json_file(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\synthesized_uid_reference.json")

In [45]:
records = [i for i in records if get_badread_strand_id(i) in strand_ids_synthesized]
#records = filter_junk_reads(records)
seqs = [str(i.seq) for i in records]

In [46]:

# ID reference functions

def get_badread_strand_id(record):
    return record.description.split()[1].split(',')[0]

def get_strand_reference(strand_record, strand_index=True):

    strand_id = get_badread_strand_id(strand_record)

    if strand_id in strand_ids_synthesized:
        if strand_index:
            return original_strand_ids.index(strand_ids_synthesized[strand_id])
        else: 
            return original_strands[original_strand_ids.index(strand_ids_synthesized[strand_id])]
    print("Invalid Strand ID!")
    return None

In [47]:
def get_edit_distance_matrix(strands):
    """
    Returns the edit distance matrix for the strands
    O(n^2)
    """

    n_strands = len(strands)
    edit_distance_matrix = np.zeros([n_strands, n_strands])
    for i in range(n_strands - 1):
        for j in range(i + 1, n_strands):
            edit_distance_matrix[i,j] = edit_distance_matrix[j, i] = ratio(strands[i], strands[j])

    return edit_distance_matrix

def calculate_centroid(strands):
    edit_distance_matrix = get_edit_distance_matrix(strands)

    distances = [sum(edit_distance_matrix[i, :]) for i in range(len(edit_distance_matrix))]
    return strands[distances.index(min(distances))]


def get_mean_edit_distance_cluster(edit_distance_matrix):
    distances = [sum(edit_distance_matrix[i, :]) for i in range(len(edit_distance_matrix))]
    return np.mean(distances)


In [60]:
def get_sample_statistics(records):
    """
    Given a sample of the data, I want to get the following
    1. Number of strands that don't match to shit
    2. Average number of unique matches per strand
    3. Std number of unique matches per strand
    3. Total number of good strands
    """

    rec_threshold = 0.85
    strands_by_index = np.zeros(len(original_strands))
    unmatched = 0
    junk = 0
    seqs = []
    
    for record in records:

        try:
            strand_id = utils.get_badread_strand_id(record)
            synthesized_id = strand_ids_synthesized[strand_id]
            index = original_strand_ids.index(synthesized_id)
            #index = original_strand_ids[strand_ids_synthesized[utils.get_badread_strand_id(record)]]
        except:
            continue

        seq = str(record.seq)
        revseq = str(utils.reverse_complement(seq))

        if ratio(seq, original_strands[index]) > rec_threshold:
            strands_by_index[index] += 1
            seqs.append(seq)
        elif ratio(revseq, original_strands[index]) > rec_threshold:
            strands_by_index[index] += 1
            seqs.append(revseq)
        else:
            unmatched += 1

    return unmatched, np.mean(strands_by_index), np.std(strands_by_index), seqs


In [61]:
unmatched, strands_by_index_1, strands_by_index_2, seqs = get_sample_statistics(records)

In [62]:
unmatched

4

In [94]:

# To catch the adapters
clusters, centroids = heirarchal_clustering.cluster_trivial(seqs, similarity_threshold=0.85, use_centroids=False, analysis=False)
clustered_seqs = [[seqs[i] for i in j] for j in clusters]

Total strands 283


100%|██████████| 283/283 [00:00<00:00, 93677.53it/s]

Number of clusters = 6


In [95]:
print(f"Number of clusters {len(clusters)}")
print(f"Number of clusters in the right range {len([i for i in clusters if len(i) > 12])}")

Number of clusters 6
Number of clusters in the right range 5


In [96]:
#clusters = [i for i in clusters if len(i) > 12]

In [97]:
indices = get_sort_by_sublists_length(clusters)
sorted_clusters = [clusters[i] for i in indices]
sorted_centroids = [centroids[i] for i in indices]
sorted_clustered_seqs = [clustered_seqs[i] for i in indices]

centroids = sorted_centroids
clusters = sorted_clusters
clustered_seqs = sorted_clustered_seqs

In [98]:
[len(i) for i in clustered_seqs]

[70, 59, 56, 50, 47, 1]

Levenshtien edit distance works pretty well. That's my metric. Kmeans works for now. I'll bring down junk reads etc, remove adapters, and test it again to see how well aggregation works.
Maybe try DBscan with Levenshtien distance

In [101]:
# And then merge and validate for top 5

original_strand_guessed_best = []

for ind, i in enumerate(centroids):
    
    best_guessed = 0
    best_rec = 0.0
    for k, original_strand in enumerate(original_strands):
        rec = ratio(i, original_strand)

        if rec > best_rec:
            best_guessed = k
            best_rec = rec
        
        rec = ratio(utils.reverse_complement(i), original_strand)

        if rec > best_rec:
            best_guessed = k
            best_rec = rec
        

    original_strand_guessed_best.append(best_guessed)
    



    print(f"{best_guessed} {best_rec} recovered by {ind}".format())
    #print(len(i))
    guess = heirarchal_clustering.make_prediction(clustered_seqs[ind], sample_size=10)
    print(ratio(guess, original_strands[best_guessed]))
    print(get_recovery_percentage(guess, original_strands[best_guessed]))
    print()


    #print(f"{len(clusters[ind])} elements in the cluster")
    #print(f"{strand_filtering.sum_entropies(i)} long run")
    #print(f"{np.mean([len(i) for i in clustered_seqs[ind]])} mean length of strand in cluster")
    #print(f"{np.std([len(i) for i in clustered_seqs[ind]])} mean std of strand in cluster")
    #print(f"{get_mean_edit_distance_cluster(distance_matrices[ind])} mean edit distance within cluster")
    #print()

1 0.9406175771971497 recovered by 0
0.9280742459396751
0.27

4 0.9586374695863747 recovered by 1
0.9501187648456058
0.225

3 0.9582309582309583 recovered by 2
0.9345794392523364
0.215

2 0.9304556354916067 recovered by 3
0.9195402298850575
0.265

0 0.9408983451536643 recovered by 4
0.9324009324009324
0.29

2 0.9095238095238095 recovered by 5


ValueError: Sample larger than population or is negative

In [109]:
clusters_2 = heirarchal_clustering.cluster_trivial(centroids, similarity_threshold=0.6)

13it [00:00, 1182.65it/s]


In [136]:
clustered_seqs.sort(key=len, reverse=True)


In [137]:
# And then merge and validate for top 5

original_strand_guessed_best = []

for i in range(len(clustered_seqs)):
    if len(clustered_seqs) > 15:
        guess = heirarchal_clustering.make_prediction(clustered_seqs[i], sample_size=15)
    else:
        guess = heirarchal_clustering.make_prediction(clustered_seqs[i], sample_size=len(clustered_seqs[i]))
    
    best_guessed = 0
    best_rec = 0.0
    for k, original_strand in enumerate(original_strands):
        rec = get_recovery_percentage(guess, original_strand)
        rec = align(guess, original_strand)

        if rec > best_rec:
            best_guessed = k
            best_rec = rec

    original_strand_guessed_best.append(best_guessed)
    print(f"{best_guessed} {best_rec} recovered by {i}".format())
    print()

3 0.5110294117647058 recovered by 0

3 1.0 recovered by 1

1 1.0 recovered by 2

0 0.5243445692883895 recovered by 3

4 1.0 recovered by 4

2 1.0 recovered by 5

0 0.5183823529411765 recovered by 6

0 0.4856115107913669 recovered by 7

0 1.0 recovered by 8

4 0.5054545454545455 recovered by 9

1 0.9900497512437811 recovered by 10

4 0.46875 recovered by 11

1 0.3701657458563536 recovered by 12

